#### Для начала посмотрим на данные и проведем необходимую работу с ними

In [29]:
import pandas as pd
df = pd.read_csv("C:/Users/Konstantin/Downloads/archive (1)/df2021-08-03.csv")
df

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01T13:02:48+0300,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09T08:13:01+0300,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21T15:24:16+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст..."
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T12:57:57+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31T17:10:49+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,Middle Software Developer C++/Python (relocati...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-16T13:45:47+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Полный день,Python;Git;Linux;iOS;C++;,Parallels,Москва,"Overview:Parallels Inc., a global leader in cr..."
1696,Python Tech Lead,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T14:40:13+0300,False,300000.0,NaN,RUR,От 1 года до 3 лет,Полный день,NaN,Финтех Айкью,Новосибирск,"Наша команда ищет тех. лида, задачами которого..."
1697,Senior Software Developer C++/Python (relocati...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-16T13:45:46+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Полный день,Python;Git;Linux;C++;x86;Windows;Unix;Virtuali...,Parallels,Москва,"Overview:Parallels Inc., a global leader in cr..."
1698,"Full-stack Developer (React, Python)",https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02T16:36:09+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Git;PostgreSQL;Python;Blockchain;Crypto;React ...,Qenetex,Москва,"Мы – финтех компания, приглашаем вас принять у..."


In [16]:
# Для начала посмотрим, какие значения содержаться в колонке 'currency'. 
df['currency'].unique()

array(['RUR', nan, 'EUR', 'USD'], dtype=object)

In [17]:
#Строки со значением NaN в столбце 'currency' можно отбросить
df = df[df['currency'].notna()]

In [18]:
#Переведем значения валют в рубля, для этого будем обращаться через Api центробанка и получать курс валюты на дату создания вакансии
#Благодаря ранее выполненному коду, мы знаем, что в качестве валюты могут быть рубли, евро и доллары
#Valute ID для евро ="R01239", для доллара США = "R01235"
#Переведем данные в столбце 'created' в формат y-m-d для отправки запроса
df['created'] = pd.to_datetime(df['created']).dt.strftime('%d/%m/%Y')



In [19]:
import requests
import xml.etree.ElementTree as ET
import numpy as np
#Функция request_api принимает дату и валюту из датафрейма и возвращает значение курса
def request_api(date, currency):
    url = 'http://www.cbr.ru/scripts/XML_daily.asp?date_req=' + date
    response = requests.get(url)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
    
        valutes = root.findall('Valute')
    
        for valute in valutes:
            if(currency == 'USD'):
                if valute.attrib.get('ID') == 'R01235':
                    for child in valute:
                        if child.tag == 'Value':
                            desired_value = child.text
                            desired_value = desired_value.replace(',', '.')
                            return float(desired_value)
            if(currency == 'EUR'):
                if valute.attrib.get('ID') == 'R01239':
                    for child in valute:
                        if child.tag == 'Value':
                            desired_value = child.text
                            desired_value = desired_value.replace(',', '.')
                            return float(desired_value)
    else:
        print('Ошибка при отправке запроса')

#Функция convert_salary принимает строку датафрейма, вызывает функцию request_api и  переводит значение зарплаты в рубли
def convert_salary(salary_from, salary_to, exchange_rate):
    if pd.notna(salary_from) and pd.notna(salary_to):
        new_sal_from = salary_from * exchange_rate
        new_sal_to = salary_to * exchange_rate
        return new_sal_from, new_sal_to
    elif pd.notna(salary_from):
        new_sal_from = salary_from * exchange_rate
        return new_sal_from, np.nan
    elif pd.notna(salary_to):
        new_sal_to = salary_to * exchange_rate
        return np.nan, new_sal_to
    else:
        return np.nan, np.nan



In [21]:
#переведем значения валют в рубли
for index, row in df.iterrows():
    date = row['created']
    salary_from = row['salary_from']
    salary_to = row['salary_to']
    currency = row['currency']
    if (currency != 'RUR'):
        currency_rate = request_api(date, currency)
        value = convert_salary(salary_from, salary_to, currency_rate)
        df.at[index, 'salary_from'] = value[0]
        df.at[index, 'salary_to'] = value[1]

#### Найдём вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [22]:
df['experience'].unique()

array(['От 1 года до 3 лет', 'От 3 до 6 лет', 'Более 6 лет', 'Нет опыта'],
      dtype=object)

In [23]:
#Переведем значения опыта из категориальных переменных в числовые
df['experience_num'] = df['experience'].apply(lambda x: 1 if x == 'От 1 года до 3 лет' else (3 if x == 'От 3 до 6 лет' else (6 if x == 'Более 6 лет' else 0)))

#### Выведем рейтинг (топ-30) навыков по всем вакансиям.

In [25]:
skills_split = df['skills'].str.split(';', expand=True).stack()
top_skills = skills_split.value_counts().head(30)
print(top_skills)


                    540
Python              520
Git                 249
PostgreSQL          201
Django Framework    198
SQL                 171
Linux               163
Docker               69
MySQL                69
Redis                68
Flask                63
JavaScript           57
ООП                  56
MongoDB              53
Английский язык      43
REST                 38
Django               37
RabbitMQ             36
HTML                 25
Базы данных          24
CSS                  24
Celery               20
React                18
API                  18
C++                  18
PHP                  17
Atlassian Jira       16
Работа в команде     16
Nginx                15
NoSQL                15
Name: count, dtype: int64


#### Cоставим топ городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [26]:
qa_vacancies = df[df['vacancy'].str.contains('QA', case=False)]
top_cities = qa_vacancies.groupby('area')[['salary_from', 'salary_to']].mean().mean(axis=1).nlargest(10)

print(top_cities)


area
Тамбов             200000.000000
Санкт-Петербург    180267.857143
Москва             175000.000000
Казань              90000.000000
dtype: float64
